# packages and imports

In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statistics import mean

from data_makers import *
import utils
from utils import *
import mean_model
from mean_model import meanModel
import ML_models
from ML_models import *

import sklearn
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, r2_score

import lime
import lime.lime_tabular

import alibi
from alibi.explainers import AnchorTabular
from alibi.explainers import CEM

import re

import nikhil_mean_model

from Nikhil_code import data_preprocessing as dp_nb
from Nikhil_code import model_selection as ms_nb
from Nikhil_code import testing as t_nb
import Nikhil_code.Data_imports as di_nb
import Nikhil_code.pairs_train_test_split as tts_nb
import Nikhil_code.Learning_curve as lc_nb

2023-10-06 11:56:43.870664: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-06 11:56:45.435647: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-06 11:56:46.240453: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-06 11:56:46.240491: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

# Feature selection (FS) and data creating for each drug

In [3]:
#input data
prot, rna, phospho_ls, one_hot_cls, one_hot_drugs, ic50_df1 = di_nb.read_input_data()
_all_cls = prot.index
_all_drugs = ic50_df1.columns
assert prot.shape[0] == rna.shape[0] == phospho_ls.shape[0]
assert phospho_ls.shape[0]  == one_hot_cls.shape[0]
prot.shape, rna.shape, phospho_ls.shape, one_hot_cls.shape, one_hot_drugs.shape

Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


Number of missing prot values 0.386335609896865
num non overlapping prot and target cls: 10


Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


num non overlapping cls: 930


((38, 8457), (38, 17417), (38, 22804), (38, 38), (345, 345))

In [4]:
## PHOSPHO
landmark_genes = pd.read_csv(
    f'data/Landmark_genes_LINCS.txt',sep='\t')
landmark_genes.index = landmark_genes['Symbol']

X_L1000 = landmark_X_maker(phospho_ls, landmark_genes['Symbol'])

#create prot data for all drugs
x_all_phos, x_drug, y_main_phos = dp_nb.create_all_drugs(
    phospho_ls, one_hot_drugs, ic50_df1, _all_cls)

#fmt index to include drug cell line paris
cls_drugs_index = x_all_phos.index + '::' + x_drug.index 
x_all_phos.index = cls_drugs_index
y_main_phos.index = cls_drugs_index
x_drug.index = cls_drugs_index

x_all_phos = x_all_phos.astype(np.float32)
print(x_all_phos.shape)
X_main_phos = X_main_maker(x_all_phos, x_drug, short = False)
print(X_main_phos.shape)

(11583, 22804)
(11583, 23149)


In [6]:
##PROTEOMIC
#use the same landmark genes for fs with prot data
landmark_genes = pd.read_csv(
    f'data/Landmark_genes_LINCS.txt',sep='\t')
landmark_genes.index = landmark_genes['Symbol']

#find overlapping landmark genes and prot features
overlapping_landmarks, _ = dp_nb.keep_overlapping(
    pd.DataFrame(landmark_genes['Symbol']), prot.T)

overlapping_landmarks = overlapping_landmarks['Symbol'].values

#create prot data for all drugs
x_all_prot, x_drug, y_list = dp_nb.create_all_drugs(
    prot, one_hot_drugs, ic50_df1, _all_cls)

#fmt index to include drug cell line paris
cls_drugs_index = x_all_prot.index + '::' + x_drug.index 
x_all_prot.index = cls_drugs_index
y_list.index = cls_drugs_index
x_drug.index = cls_drugs_index

x_all_prot = x_all_prot.astype(np.float32)
x_all_prot.shape

(11583, 8457)

In [7]:
#one hot data creation for all drugs
x_hot, x_drug_hot, y_main_onehot = dp_nb.create_all_drugs(
    one_hot_cls, one_hot_drugs, ic50_df1, _all_cls)

cls_drugs_index_hot = x_hot.index + '::' + x_drug_hot.index 

x_hot.index = cls_drugs_index_hot
x_hot.columns = np.arange(len(x_drug.columns), len(x_drug.columns) + len(x_hot.columns))
print(x_hot.shape)
X_main_onehot = X_main_maker(x_hot, x_drug, short = False)
print(X_main_onehot.shape)

(11583, 38)
(11583, 383)


# Random Features

In [ ]:
#create dataframes that take a random set of features with the same length as landmark as a comparison
random_X = X_phos.sample(n=2056,axis='columns')

In [ ]:
#make the model a classifier
y_main_phos = classyFire(y_main_phos)

# Feature Selection

In [8]:
#read in the feature_importances_ feature selected data
feature_list = []
with open("feat_select_files/phospho/rfr_X_main/rfr_feat_select.txt", "r") as features:
    lines = features.readlines()
    for i in lines:
        i.replace(" ", "")
        feature_list.append(i.split(":")[0])

In [11]:
#create a new X dataframe with the selected features
X_features = phospho_ls.reindex(feature_list[:2056],axis="columns")

In [12]:
#create prot data for all drugs
x_all_phos, x_drug, y_main_phos = dp_nb.create_all_drugs(
    X_features, one_hot_drugs, ic50_df1, _all_cls)

#fmt index to include drug cell line paris
cls_drugs_index = x_all_phos.index + '::' + x_drug.index 
x_all_phos.index = cls_drugs_index
y_main_phos.index = cls_drugs_index
x_drug.index = cls_drugs_index

x_all_phos = x_all_phos.astype(np.float32)
print(x_all_phos.shape)
X_main_phos = X_main_maker(x_all_phos, x_drug, short = False)
print(X_main_phos.shape)

(11583, 2056)
(11583, 2401)


In [ ]:
#make the model a classifier
y_main_phos = classyFire(y_main_phos)

# first train-test split

N.B. X_train and y_train replace X_main and y_main

In [13]:
#regular split
X_train, X_test, y_train, y_test = cell_line_split(X_main_phos, y_main_phos, test_size=0.2, random_state = 0)

In [ ]:
#short split
X_train_short, X_test_short, y_train_short, y_test_short = cell_line_split(X_main_phos[:1000], y_main_phos[:1000], test_size=0.2, random_state = 0)

# Benchmark mean model 

create a model that predicts IC50 by looking at mean IC50 value for that drug

In [ ]:
dl = dlMaker(y_train, noRepeats=True)

In [ ]:
mm = meanModel(y_train, dl)

In [ ]:
prediction = mm.predict(y_test.index)

In [ ]:
r2_score(y_test, list(prediction.values()))

# Nikhil mean model

In [ ]:
dl = dlMaker(y_train, noRepeats=True)

In [ ]:
nmm = MeanModel(y_train, dl)

In [ ]:
prediction = nmm.predict(y_test.index)

In [ ]:
r2_score(y_test, list(prediction))

# Mixed-set benchmark model

In [ ]:
#random forests
r2, MSE, pearson, SE = rfr(X_train, y_train, test_size=0.2, random_state=0, iterations=3)

# Random Forest 

## regressor

In [ ]:
rs = [0, 8, 23, 42, 69, 88]
 
mm_r2 = []
mixed_set_r2 = []
model_r2 = []
model_standard_error = []
model_MSE = []
model_pearson = []

for seed in rs:
    print(f'Seed: {seed}')
    
    print(f'Mean Model')
    #MEAN MODEL
    #tts
    X_train, X_test, y_train, y_test = cell_line_split(X_main_phos, y=y_main_phos, test_size=0.2, random_state = seed)
    #run mean model
    dl = dlMaker(y_train, noRepeats=True)
    mm = meanModel(y_train, dl)
    prediction = mm.predict(y_test.index)
    mean_r2 = r2_score(y_test, list(prediction.values()))
    mm_r2.append(mean_r2)
    
    print(f'Mixed-Set Model')
    #MIXED SET
    # one hot the dataframes
    X_main_onehot, y_main_onehot = mixed_set_maker(X_L1000, y_phos)
    #MODEL
    r2, MSE, pearson, standard_error = rfr(X_main_onehot, y_main_onehot, test_size=0.2, random_state=seed, iterations=3)
    mixed_set_r2.append(r2)

    print(f'Model: rfr')
    #MODEL
    r2, MSE, pearson, standard_error = rfr(X_main_phos, y_main_phos, test_size=0.2, random_state=seed, iterations=3)

    model_r2.append(r2)
    model_standard_error.append(standard_error)
    model_MSE.append(MSE)
    model_pearson.append(pearson)

In [ ]:
table_make('rfr_metrics',
           [rs, mm_r2, mixed_set_r2, model_r2, model_standard_error, model_MSE, model_pearson],
           ['rs', 'mm_r2', 'mixed_set_r2', 'model_r2', 'model_standard_error', 'model_MSE', 'model_pearson'], 
           file = 'plots/phospho/NormalisedX_rfr_metrics_L1000.csv')
           

## classifier

In [ ]:
rs = [0, 8, 23, 42, 69, 88]
 
mixed_set_accuracy = []
model_accuracy = []

for seed in rs:
    print(f'Seed: {seed}')
      
    print(f'Mixed-Set Model')
    #MIXED SET
    # one hot the dataframes
    X_main_onehot, y_main_onehot = mixed_set_maker(X_phos, y_phos)
    #make the model a classifier
    y_main_onehot = classyFire(y_main_onehot)
    #MODEL
    predictions, accuracy = classy_rfr(X_main_onehot, y_main_onehot, test_size=0.2, random_state=seed, iterations=3)
    mixed_set_accuracy.append(accuracy)

    print(f'Model: rfr')
    #MODEL
    predictions, accuracy = classy_rfr(X_main_phos, y_main_phos, test_size=0.2, random_state=seed, iterations=3)

    model_accuracy.append(accuracy)


In [ ]:
table_make('rfr_metrics',
           [rs, mixed_set_accuracy, model_accuracy],
           ['rs', 'mixed_set_accuracy', 'model_accuracy'], 
           file = 'plots/phospho/CLASSY_NormalisedX_rfr_metrics.csv')
           

# feature selection with rfr feature_importances

## feature selection

Here I run the models and store the feature_importances_ data

In [ ]:
#run the model that outputs feature_importances_ attribute with full length
r2, MSE, classify = rfrFeatSelect(X = X_main, y = y_main, random_state = 88, iterations=1)
print('R-squared: '+str(r2)+'\n'+'Mean squared error: '+str(MSE))

# classify is the feature_importances_ array
print(classify)

In [ ]:
#this function outputs the top x number of features and their scores for a model
rfr_final_names, rfr_final_scores = rfrFeatures(classify, X_main = X_main, topX = 10411, N = 10411)

In [ ]:
# assign plot name variable for saving plots
plot_name = 'plots/rfr_proteomic_rs88.png'

In [ ]:
#plot the feature_importances_

plt.rcParams['figure.figsize'] = [20, 20]
plt.plot(rfr_final_names[:30], rfr_final_scores[:30], linestyle='-', marker='.', color='#009d9a', linewidth=1)
rot = plt.xticks(rotation=45)
plt.savefig(plot_name)

In [ ]:
# creates a dictionary of features in order of importance with their score to store in a text file
rfrdict = {rfr_final_names[i]:rfr_final_scores[i] for i in range(len(rfr_final_names))}

In [ ]:
# specify text file name
file_name = 'feat_select_files/proteomic/rfr_feat_select_proteomic_rs88.txt'

In [ ]:
#store all feature_importance_ date in a text file

with open(file_name, "w") as txt_file:
    for key, value in rfrdict.items():
        txt_file.write(key +':'+ str(value) + "\n") 

In [ ]:
# Import data from the text files and make final_names/final_scores from feature txt file
rfr_final_names = []
rfr_final_scores = []
with open(file_name, "r") as features:
    lines = features.readlines()
    for i in lines:
        phospho = i.split(':')[0]
        score = i.split(':')[1]
        score = score.split("\n")[0]
        rfr_final_names.append(phospho)
        rfr_final_scores.append(float(score))

In [ ]:
#plot this data

plt.rcParams['figure.figsize'] = [20, 20]
plt.plot(rfr_final_names[:50], rfr_final_scores[:50], linestyle='-', marker='.', color='#009d9a', linewidth=1)
rot = plt.xticks(rotation=45)
plt.savefig('plots/rfr_1.png')

## feature selection testing

Test how effectively the model runs with various numbers of features from the top x feature_importances_ outputs

In [ ]:
#read in the feature_importances_ feature selected data
feature_list = []
with open("feat_select_files/phospho/rfr_X_main/rfr_feat_select.txt", "r") as features:
    lines = features.readlines()
    for i in lines:
        i.replace(" ", "")
        feature_list.append(i.split(":")[0])

In [ ]:
seeds = [0, 8, 23, 42, 69, 88]
for i in range(5,10):
    index = i*100
    for s in range(len(seeds)):
        #create the dataframes
        X_features = X_phos.reindex(feature_list[:index],axis="columns")
        hotdrugsDF_feats = one_hot_maker(y_phos)
        cl_phos = clMaker(X_phos, y_phos)
        x_all_feats, x_drug_feats, y_main_feats = create_all_drugs(x=X_features, xd=hotdrugsDF_feats, y=y_phos, cells=cl_phos)
        X_main_feats = X_main_maker(x_all_feats, x_drug_feats, short = False)
        X_train_features, X_test_features, y_train_features, y_test_features = cell_line_split(X_main_feats, y_main_feats, test_size=0.2, random_state = seeds[s])
        
        #status update print statements
        print(f'number of features: {index}, seed: {seeds[s]}')
        
        #run the model
        r2, MSE, pearson = rfr(X = X_train_features, y = y_train_features, test_size=0.2, random_state = seeds[s], iterations=3)
        print('R-squared: '+str(r2)+'\n'+'Mean squared error: '+str(MSE))
    

# feature selection with permutation_importance

read in feature_importances data and test on this random forest method to see if they agree

In [21]:
X_train, X_test, y_train, y_test = cell_line_split(X_main_phos, y_main_phos, test_size=0.2, random_state = 0)
print(f'X_train.shape: {X_train.shape}, X_test.shape: {X_test.shape}')

X_train.shape: (9009, 2401), X_test.shape: (2574, 2401)


In [19]:
%%time
# permutation importance- fit model
classify = RandomForestRegressor(n_jobs=-1, max_depth=300, n_estimators=200)
classify.fit(X_main_phos.values, y_main_phos)

CPU times: user 1h 23min 43s, sys: 1 s, total: 1h 23min 44s
Wall time: 12min 4s


RandomForestRegressor(max_depth=300, n_estimators=200, n_jobs=-1)

In [22]:
%%time
# permutation importance
result = permutation_importance(classify, X_main_phos.values, y_main_phos, n_repeats=10, random_state=0)

CPU times: user 12h 55min, sys: 13min 53s, total: 13h 8min 53s
Wall time: 2h 42min 48s


In [ ]:
#read in the feature_importances_ feature selected data
feature_list = []
with open("feat_select_files/phospho/rfr_X_main/rfr_feat_select.txt", "r") as features:
    lines = features.readlines()
    for i in lines:
        i.replace(" ", "")
        feature_list.append(i.split(":")[0])

In [23]:
#this function outputs the top x number of features and their scores for a model

rfr_final_names, rfr_final_scores = topFeatures(result.importances_mean, X_main = X_main_phos, topX = 1000, N = 1000)

In [ ]:
#plot the data
plt.rcParams['figure.figsize'] = [20, 20]
plt.plot(rfr_final_names[:50], rfr_final_scores[:50])
rot = plt.xticks(rotation=45)

# LIME for rfr

In [ ]:
#read in the feature_importances_ feature selected data
feature_list = []
with open("feat_select_files/phospho/rfr_X_main/rfr_feat_select.txt", "r") as features:
    lines = features.readlines()
    for i in lines:
        i.replace(" ", "")
        feature_list.append(i.split(":")[0])

In [ ]:
# filter the first 1000 feature_importances_
X_features = X_phos.reindex(feature_list[:1000],axis="columns")

#create the remaining dataframes
hotdrugsDF_feats = one_hot_maker(y_phos)
cl_phos = clMaker(X_features, y_phos)
x_all_feats, x_drug_feats, y_main_feats = create_all_drugs(x=X_features, xd=hotdrugsDF_feats, y=y_phos, cells=cl_phos)
X_main_feats = X_main_maker(x_all_feats, x_drug_feats, short = False)


In [ ]:
#train-test-split
X_train_features, X_test_features, y_train_features, y_test_features = cell_line_split(X_main_feats, y_main_feats, test_size=0.2, random_state = 0)

In [ ]:
print(f'X_train shape: {X_train_features.shape}\ny_train shape: {y_train_features.shape}')

In [ ]:
#run the model
rfr = RandomForestRegressor(n_jobs=-1, max_depth=300, n_estimators=200)
rfr.fit(X_train_features, y_train_features)

# predict_fn function

def predict_FN(data):
    # Perform predictions using your regressor model
    predictions = rfr.predict(data)
    return predictions

In [ ]:
# create the LIME explainer

explainer = lime.lime_tabular.LimeTabularExplainer(X_train_features.to_numpy(),
                                                   mode='regression',
                                                   training_labels=y_train_features,
                                                   feature_names=list(X_train_features.columns))

In [ ]:
%%time
exp = explainer.explain_instance(X_train_features.iloc[1].values, predict_FN, num_features=1000)

In [ ]:
exp_list = exp.as_list()

In [ ]:
# remove drug features
feat_list = []
for i, LIME in enumerate(exp_list):
    matches = re.findall(r'[A-Z]', LIME[0])
    if matches:
        feat_list.append(f'{LIME[0]}: {LIME[1]}')
        
feat_list[:10]

## LIME for xgb

In [ ]:
#read in the feature_importances_ feature selected data
feature_list = []
with open("feat_select_files/phospho/rfr_X_main/rfr_feat_select.txt", "r") as features:
    lines = features.readlines()
    for i in lines:
        i.replace(" ", "")
        feature_list.append(i.split(":")[0])

In [ ]:
# filter the first 1000 feature_importances_
X_features = X_phos.reindex(feature_list[:1000],axis="columns")

#create the remaining dataframes
hotdrugsDF_feats = one_hot_maker(y_phos)
cl_phos = clMaker(X_features, y_phos)
x_all_feats, x_drug_feats, y_main_feats = create_all_drugs(x=X_features, xd=hotdrugsDF_feats, y=y_phos, cells=cl_phos)
X_main_feats = X_main_maker(x_all_feats, x_drug_feats, short = False)


In [ ]:
#train-test-split
X_train_features, X_test_features, y_train_features, y_test_features = cell_line_split(X_main_feats, y_main_feats, test_size=0.2, random_state = 0)

In [ ]:
print(f'X_train shape: {X_train_features.shape}\ny_train shape: {y_train_features.shape}')

In [ ]:
#run the model
xgb = XGBRegressor(max_depth = 75, 
                   n_estimators = 300, 
                   seed = 42, 
                   min_child_weight = 3, 
                   gamma = 0, 
                   colsample_bytree = 0.3, 
                   reg_alpha = 0.1,
                   n_jobs=-1)

xgb.fit(X_train_features, y_train_features)

# predict_fn function

def predict_FN(data):
    # Perform predictions using your regressor model
    predictions = xgb.predict(data)
    return predictions

In [ ]:
# create the LIME explainer

explainer = lime.lime_tabular.LimeTabularExplainer(X_train_features.to_numpy(),
                                                   mode='regression',
                                                   training_labels=y_train_features,
                                                   feature_names=list(X_train_features.columns))

In [ ]:
%%time
exp = explainer.explain_instance(X_train_features.iloc[1].values, predict_FN, num_features=1000)

In [ ]:
exp_list = exp.as_list()

In [ ]:
# remove drug features
feat_list = []
for i, LIME in enumerate(exp_list):
    matches = re.findall(r'[A-Z]', LIME[0])
    if matches:
        feat_list.append(f'{LIME[0]}: {LIME[1]}')
        
feat_list[:10]

# Anchor explanations

In [ ]:
#train-test-split
X_train, X_test, y_train, y_test = cell_line_split(X_main_phos, y_main_phos, test_size=0.2, random_state = 0)

In [ ]:
#run the model
rfr = RandomForestClassifier(n_jobs=-1, max_depth=300, n_estimators=200)
rfr.fit(X_train, y_train)

In [ ]:
# predict function
predict_fn = lambda x: rfr.predict_proba(x)

In [ ]:
# create the explainer
explainer = AnchorTabular(predict_fn, list(X_train.columns))

In [ ]:
#discretise the features into quartiles
explainer.fit(X_train.to_numpy(), disc_perc=(25, 50, 75))

In [ ]:
#get the anchor
idx = 0
class_names = [0, 1, 2]
X_test = X_test.to_numpy()
print('Prediction: ', class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]])

In [ ]:
#set the precision threshold

explanation = explainer.explain(X_test[idx], threshold=0.95)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

# Contrastive explanation model

In [ ]:
pip install --upgrade tensorflow==2.10

In [ ]:
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs

In [ ]:
# set parameters and train
num_epochs = 200 # max epoch for training
learning_rate = 1e-2 # initial learning rate, decays via learning rate scheduler
momentum = 0.5
initializer_seed = 42
 
model = DeepIC50(X_train, learning_rate, momentum, initializer_seed, mtype='classifier')

In [ ]:
#run the model
model = XGBClassifier(max_depth = 75, 
                   n_estimators = 300, 
                   seed = 42, 
                   min_child_weight = 3, 
                   gamma = 0, 
                   colsample_bytree = 0.3, 
                   reg_alpha = 0.1,
                   n_jobs=-1)

In [ ]:
# initialize CEM explainer
shape = X_train.shape
mode = 'PN'
cem = CEM(model, mode, shape, kappa=0., beta=.1,
          feature_range=(X_train.min(), X_train.max()),
          gamma=100, max_iterations=1000,
          c_init=1., c_steps=10, learning_rate_init=1e-2,
          clip=(-1000.,1000.), no_info_val=-1.)